# Exploring industry returns in the Fama-French data-set

The [49 Industry Portfolios](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_49_ind_port.html) data-set has a series of daily returns of 49 different industry types. Here, we look at some of their statistical properties.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
startDt <- as.Date("1990-01-01")

#value (market-cap) weighted
valueWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

#equal weighted
equalWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AEWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

In [3]:
lookbackDays <- 220 * 5

rrValueWtd <- rollapply(valueWtd, lookbackDays, Return.cumulative, by.column = F)
rrEqualWtd <- rollapply(equalWtd, lookbackDays, Return.cumulative, by.column = F)

rrValueWtd <- na.omit(rrValueWtd)
rrEqualWtd <- na.omit(rrEqualWtd)

In [4]:
minValWtd <- apply(rrValueWtd, 2, min)
minEqlWtd <- apply(rrEqualWtd, 2, min)

maxValWtd <- apply(rrValueWtd, 2, max)
maxEqlWtd <- apply(rrEqualWtd, 2, max)

sdValWtd <- apply(rrValueWtd, 2, sd)
sdEqlWtd <- apply(rrEqualWtd, 2, sd)

print("MIN")
print(minValWtd)
print(minEqlWtd)

print("MAX")
print(maxValWtd)
print(maxEqlWtd)

print("SD")
print(sdValWtd)
print(sdEqlWtd)

[1] "MIN"
       X.1        X.2        X.3        X.4        X.5        X.6        X.7 
-0.2532453 -0.4074237 -0.7431826 -0.7484332 -0.2183618 -0.5767641 -0.7936408 
       X.8        X.9       X.10       X.11       X.12       X.13       X.14 
-0.4499764 -0.4315746 -0.3609480 -0.7657270 -0.3782547 -0.5927093 -0.9631609 
      X.15       X.16       X.17       X.18       X.19       X.20       X.21 
-0.2876000 -0.3043374 -0.6453013 -0.6100470 -0.1204453 -0.6996230 -0.7739683 
      X.22       X.23       X.24       X.25       X.26       X.27       X.28 
-0.3086047 -0.7421280 -0.3694879 -0.2153368 -0.5480310 -0.5442701 -0.3163298 
      X.29       X.30       X.31       X.32       X.33       X.34       X.35 
-0.2264927 -0.2991468 -0.5143283 -0.3595542 -0.5805491 -0.5082174 -0.3867057 
      X.36       X.37       X.38       X.39       X.40       X.41       X.42 
-0.7545607 -0.2353749 -0.4900953 -0.3139155 -0.2988185 -0.4583180 -0.6970507 
      X.43       X.44       X.45       X.46       X.47

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)